Loading the .env file

In [123]:
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path=r"D:\Lang_chain\.env")

True

Importing google api key for our model

In [124]:
os.environ["GOOGLE_API_KEY"]=os.getenv("google_ai_api_key")

Tracking llm in langsmith

In [125]:
os.environ["LANGSMITH_TRACING"]="true"
os.environ["LANGSMITH_API_KEY"]=os.getenv("langsmith_api_key")
os.environ["LANGSMITH_PROJECT"]="Q&A Chatbot"

In [126]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash-thinking-exp-01-21")

llm

ChatGoogleGenerativeAI(model='models/gemini-2.0-flash-thinking-exp-01-21', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000218E585D310>, default_metadata=())

Url document loader

In [127]:
from langchain_community.document_loaders import WebBaseLoader

loader=WebBaseLoader(web_path=r"https://en.wikipedia.org/wiki/Prabhas")

loaded=loader.load()

loaded

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Prabhas', 'title': 'Prabhas - Wikipedia', 'language': 'en'}, page_content='\n\n\n\nPrabhas - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nE

text splitting

In [128]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

splitted=splitter.split_documents(loaded)

splitted

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Prabhas', 'title': 'Prabhas - Wikipedia', 'language': 'en'}, page_content='Prabhas - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nEarly lif

In [129]:
len(splitted)

58

vector embeddings

In [130]:
os.environ['HF_TOKEN']=os.getenv("hf_api_key")

In [131]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

embedding

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

vectordb

In [132]:
from langchain_community.vectorstores import FAISS

db=FAISS.from_documents(documents=splitted,embedding=embedding)

db

retriever

In [133]:
retreiver=db.as_retriever(search_kwargs={'k': 1})

retreiver

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000218E5DC5220>, search_kwargs={'k': 1})

creating history aware retrieval chain

In [134]:
from langchain_community.chat_message_histories import ChatMessageHistory

sessions={}

def get_session_info(session_id):

    if session_id not in sessions:

        sessions[session_id]=ChatMessageHistory()

    return sessions[session_id]



In [135]:
configuration={"configurable":{"session_id":"1"}}

Importing runnables to run using sessioids

In [136]:
from langchain_core.messages import trim_messages

trim=trim_messages(
    max_tokens=100,
    strategy="last",
    token_counter=llm,
    start_on="human",
    include_system=True,
    allow_partial=False
)


In [137]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

pas=RunnablePassthrough.assign(memory=itemgetter("memory")|trim)

pas


RunnableAssign(mapper={
  memory: RunnableLambda(itemgetter('memory'))
          | RunnableLambda(...)
})

In [138]:
from langchain_core.prompts import MessagesPlaceholder

In [139]:
from langchain.chains import create_history_aware_retriever,create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

template=ChatPromptTemplate(
    [
        ("system","You are an Question- Answering chatbot.For answering the question if it needs previous memory then using the memory understand and generate a relavent question that matches with the given input question"),
        MessagesPlaceholder(variable_name="memory"),
        ("user","{input}")
    ]
)

template

ChatPromptTemplate(input_variables=['input', 'memory'], input_types={'memory': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_cor

In [140]:
retreieval_chain_with_history=create_history_aware_retriever(llm,retreiver,template)

retreieval_chain_with_history

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000218E5DC5220>, search_kwargs={'k': 1}))], default=ChatPromptTemplate(input_variables=['input', 'memory'], input_types={'memory': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessa

In [141]:
retreival_chain=pas|retreieval_chain_with_history

retreival_chain

RunnableAssign(mapper={
  memory: RunnableLambda(itemgetter('memory'))
          | RunnableLambda(...)
})
| RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
  | VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000218E5DC5220>, search_kwargs={'k': 1}))], default=ChatPromptTemplate(input_variables=['input', 'memory'], input_types={'memory': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.to

In [142]:
template1=ChatPromptTemplate(
    [
        ("system","Include {context} while genrating the answer keep the answer in simple words donot make much complex answer"),
        MessagesPlaceholder(variable_name="memory"),
        ("user","{input}")
    ]
)

template1

ChatPromptTemplate(input_variables=['context', 'input', 'memory'], input_types={'memory': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[la

In [143]:
document_chain=create_stuff_documents_chain(llm=llm,prompt=template1)

document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input', 'memory'], input_types={'memory': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_cor

In [144]:
chain=create_retrieval_chain(retreival_chain,document_chain)

chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableAssign(mapper={
             memory: RunnableLambda(itemgetter('memory'))
                     | RunnableLambda(...)
           })
           | RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
             | VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000218E5DC5220>, search_kwargs={'k': 1}))], default=ChatPromptTemplate(input_variables=['input', 'memory'], input_types={'memory': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')]

In [145]:
from langchain_core.runnables import RunnableWithMessageHistory

run=RunnableWithMessageHistory(chain,get_session_history=get_session_info,input_messages_key="input",history_messages_key="memory",output_messages_key="answer")

run

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  memory: RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
}), kwargs={}, config={'run_name': 'insert_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]), kwargs={}, config={}, config_factories=[], get_session_history=<function get_session_info at 0x00000218E6C70040>, input_messages_key='input', output_messages_key='answer', history_messages_key='memory', history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

In [147]:
result=run.invoke(
    {
        "input":"when did Uppalapati Venkata Suryanarayana Prabhas Raju born?"
    },
    config={"configurable":{"session_id":"2"}}
)

result

{'input': 'when did Uppalapati Venkata Suryanarayana Prabhas Raju born?',
 'memory': [],
 'context': [Document(id='c613196f-6018-4f7f-bb8a-9138782b9e17', metadata={'source': 'https://en.wikipedia.org/wiki/Prabhas', 'title': 'Prabhas - Wikipedia', 'language': 'en'}, page_content='Early life and education\nPrabhas was born on 23 October 1979 in a Telugu family to film producer Uppalapati Surya Narayana Raju and Siva Kumari in Madras (now Chennai). The youngest of three children, he has an elder brother, Prabodh, and elder sister, Pragathi.[3][21][22] He is the nephew of Telugu film actor Krishnam Raju.[3] His family hails from Mogalthur, near Bhimavaram of West Godavari district, Andhra Pradesh.[23]\nPrabhas did his schooling at Don Bosco Matriculation Higher Secondary School, Chennai, and at DNR High School, Bhimavaram.[24] He then completed his intermediate education from Nalanda College, Hyderabad.[25][24] He later went on to pursue Bachelor of Technology (B.Tech) from Sri Chaitanya C

In [ ]:
get_session_info("2")

InMemoryChatMessageHistory(messages=[HumanMessage(content='What is a  large language model (LLM)?', additional_kwargs={}, response_metadata={}), AIMessage(content="Imagine a computer program that's been trained on a massive amount of text and code. That's basically what a Large Language Model (LLM) is.\n\nThink of it like a really smart student who has read almost everything on the internet.  Because it has read so much, it can:\n\n* **Understand and generate text:** It can understand what you write and write back in a way that sounds like a human.\n* **Answer questions:**  You can ask it questions, and it will try to give you helpful answers based on what it has learned.\n* **Translate languages:** It can translate text from one language to another.\n* **Write different kinds of creative content:** It can write stories, poems, emails, and even code!\n\n**In simple words:** An LLM is a powerful computer program that's really good at understanding and working with language because it ha

In [ ]:
o=run.invoke(
    {"input":"What is the previous question i asked?"},
    config={"configurable":{"session_id":"2"}}
)

o

{'input': 'What is the previous question i asked?',
 'memory': [HumanMessage(content='What is a  large language model (LLM)?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Imagine a computer program that's been trained on a massive amount of text and code. That's basically what a Large Language Model (LLM) is.\n\nThink of it like a really smart student who has read almost everything on the internet.  Because it has read so much, it can:\n\n* **Understand and generate text:** It can understand what you write and write back in a way that sounds like a human.\n* **Answer questions:**  You can ask it questions, and it will try to give you helpful answers based on what it has learned.\n* **Translate languages:** It can translate text from one language to another.\n* **Write different kinds of creative content:** It can write stories, poems, emails, and even code!\n\n**In simple words:** An LLM is a powerful computer program that's really good at understanding and workin

In [ ]:
o['answer']

'The previous question you asked was:\n\n**What is a large language model (LLM)?**'